In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

import torch
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd
from tqdm import tqdm

from utils.config import Config
from models.base import ImageCaptioningModel
from models.attention_model import ImageCaptioningWithAttention
from data.data_set import DataSet
from text.tokenizer import Tokenizer
from pathlib import Path


In [90]:
RUN_ID = 14

In [ ]:
# laod the prediction results
path_run = f"../models/checkpoints/run_{RUN_ID}"
predictions = pd.read_csv(f"{path_run}/results.csv")
predictions.sort_values(by="BLEU Score", ascending=False, inplace=True)

# select best prediction for each image
best_predictions = predictions.loc[predictions.groupby('Image Name')['BLEU Score'].idxmax()]
best_predictions.sort_values(by="BLEU Score", ascending=False, inplace=True)


MODEL_PATH = Path(f"../models/checkpoints/run_{RUN_ID}")

config = Config(experiment_id=RUN_ID)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = ImageCaptioningWithAttention(config.embed_size, config.attention_dim, config.decoder_dim, config.vocab_size, config.encoder_dim).to(device)
model.load_state_dict(torch.load(f"{MODEL_PATH}/best_model.pth", map_location=device, weights_only=True))


In [85]:
# show image with true caption and predicted caption 
def show_image(image_name):
    path = "../data/raw/images/"
    true_caption = best_predictions.loc[best_predictions['Image Name'] == image_name, 'True Caption'].values[0]
    predicted_caption = best_predictions.loc[best_predictions['Image Name'] == image_name, 'Predicted Caption'].values[0]
    bleu_score = best_predictions.loc[best_predictions['Image Name'] == image_name, 'BLEU Score'].values[0]
    plt.figure(figsize=(10, 8))
    plt.imshow(Image.open(f"{path}/{image_name}"))
    plt.title(f"True Caption: {true_caption.replace('  ', '')}\nPredicted Caption: {predicted_caption.strip()}\nBLEU Score: {bleu_score}",fontsize=12, pad=10, wrap=True)
    plt.axis('off')
    plt.show()
    
def visualize_features_overlay(image_name, model, alpha=0.4):
    """
    Visualize mean feature activation overlaid on the original image
    """
    path = "../data/raw/images/"
    image = Image.open(f"{path}/{image_name}").convert("RGB")
    with torch.no_grad():
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225])
        ])
        image = transform(image)
        features = model.encoder(image.unsqueeze(0).to(device))

        mean_features = features.mean(dim=1).squeeze().cpu()
        mean_features = (mean_features - mean_features.min()) / (mean_features.max() - mean_features.min())

        plt.figure(figsize=(10, 8))

        img_np = image.permute(1, 2, 0).cpu().numpy()
        img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())  # Normalize to [0,1]
        plt.imshow(img_np)
        
        mean_features_resized = F.interpolate(
            mean_features.unsqueeze(0).unsqueeze(0), 
            size=img_np.shape[:2], 
            mode='bilinear', 
            align_corners=False
        ).squeeze().numpy()
        
        plt.imshow(mean_features_resized, cmap='hot', alpha=alpha)
        plt.axis('off')
        plt.show()
            

In [ ]:
best_predictions.head()

In [ ]:
index = 3
image_name = best_predictions['Image Name'].iloc[index] #3356369156_074750c6cc.jpg , 2699733386_c346c87ea6.jpg

print(image_name)
show_image(image_name)
visualize_features_overlay(image_name, model)


In [ ]:
# show the results for the best prediction
# After cleaning and grouping by image name, the best prediction is the one with the highest BLEU score
best_predictions.describe()


print('Best 5 predictions:')
for i in range(5):
    print('True Caption: ', best_predictions['True Caption'].iloc[i])
    print('Predicted Caption: ', best_predictions['Predicted Caption'].iloc[i])
    print('BLEU Score: ', best_predictions['BLEU Score'].iloc[i])
    print('-'*100)
    
print('Worst 5 predictions:')
for i in range(5):
    print('True Caption: ', best_predictions['True Caption'].iloc[-i])
    print('Predicted Caption: ', best_predictions['Predicted Caption'].iloc[-i])
    print('BLEU Score: ', best_predictions['BLEU Score'].iloc[-i])
    print('-'*100)


In [ ]:
# show the histogram of the bleu score 
plt.figure(figsize=(10, 8))
plt.title('BLEU Score Distribution')
plt.xlabel('BLEU Score')
plt.ylabel('Frequency')
plt.hist(best_predictions['BLEU Score'], bins=10)
plt.show()